# Income Data By Zip Code for New York City

This code takes the IRS Individual Income Tax Statistics (2015 - data found [here](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2015-zip-code-data-soi)) and determines the relative income level of each zip code in New York City.

Let's begin with some standard setup - import of libraries and formatting.

In [1]:
from __future__ import print_function, division

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from urllib.request import urlopen

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

Let's read in the data from the IRS website:

In [3]:
df = pd.read_csv('15zpallagi.csv')

In [4]:
df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,836320.0,481570.0,109790.0,233260.0,455560.0,1356760.0,...,373410.0,328469.0,0.0,0.0,0.0,0.0,61920.0,48150.0,732670.0,1933120.0
1,1,AL,0,2,494830.0,206630.0,146250.0,129390.0,275920.0,1010990.0,...,395880.0,965011.0,0.0,0.0,0.0,0.0,73720.0,107304.0,415410.0,1187403.0
2,1,AL,0,3,261250.0,80720.0,139280.0,36130.0,155100.0,583910.0,...,251490.0,1333418.0,0.0,0.0,0.0,0.0,64200.0,139598.0,193030.0,536699.0
3,1,AL,0,4,166690.0,28510.0,124650.0,10630.0,99950.0,423990.0,...,165320.0,1414283.0,0.0,0.0,0.0,0.0,45460.0,128823.0,116440.0,377177.0
4,1,AL,0,5,212660.0,19520.0,184320.0,4830.0,126860.0,589490.0,...,212000.0,3820152.0,420.0,168.0,60.0,31.0,83330.0,421004.0,121570.0,483682.0


A guide on the IRS website helps us to make sense of the column headings and their meaning.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166680 entries, 0 to 166679
Columns: 131 entries, STATEFIPS to A11902
dtypes: float64(127), int64(3), object(1)
memory usage: 166.6+ MB


In [6]:
df.columns

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'PREP', 'N2',
       ...
       'N10300', 'A10300', 'N85530', 'A85530', 'N85300', 'A85300', 'N11901',
       'A11901', 'N11902', 'A11902'],
      dtype='object', length=131)

In [7]:
# just in case there's white space
df.columns = [column.strip() for column in df.columns]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166680 entries, 0 to 166679
Columns: 131 entries, STATEFIPS to A11902
dtypes: float64(127), int64(3), object(1)
memory usage: 166.6+ MB


In [9]:
df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,1,836320.0,481570.0,109790.0,233260.0,455560.0,1356760.0,...,373410.0,328469.0,0.0,0.0,0.0,0.0,61920.0,48150.0,732670.0,1933120.0
1,1,AL,0,2,494830.0,206630.0,146250.0,129390.0,275920.0,1010990.0,...,395880.0,965011.0,0.0,0.0,0.0,0.0,73720.0,107304.0,415410.0,1187403.0
2,1,AL,0,3,261250.0,80720.0,139280.0,36130.0,155100.0,583910.0,...,251490.0,1333418.0,0.0,0.0,0.0,0.0,64200.0,139598.0,193030.0,536699.0
3,1,AL,0,4,166690.0,28510.0,124650.0,10630.0,99950.0,423990.0,...,165320.0,1414283.0,0.0,0.0,0.0,0.0,45460.0,128823.0,116440.0,377177.0
4,1,AL,0,5,212660.0,19520.0,184320.0,4830.0,126860.0,589490.0,...,212000.0,3820152.0,420.0,168.0,60.0,31.0,83330.0,421004.0,121570.0,483682.0


We need to find the zip codes in NYC. Instead of manually listing the zip codes we're looking for, we've done it programatically, with some basic webscraping and string conversion in order to generate a list of zip codes in NYC.

In [10]:
# web scraping!
import requests
from bs4 import BeautifulSoup

url = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm"

page = urlopen(url).read()
soup = BeautifulSoup(page, "lxml")

all_tables=soup.find_all('table')

right_table=soup.find('table')
right_table

zip_codes=[]

rows = right_table.find_all('tr')
zip_string = ''
for tr in rows:
    cols = tr.find_all('td')
    for td in cols:
        zip_string += td.text

In [11]:
zip_string

'Bronx Central Bronx 10453, 10457, 10460 Bronx Park and Fordham 10458, 10467, 10468 High Bridge and Morrisania 10451, 10452, 10456 Hunts Point and Mott Haven 10454, 10455, 10459, 10474 Kingsbridge and Riverdale 10463, 10471 Northeast Bronx 10466, 10469, 10470, 10475 Southeast Bronx 10461, 10462,10464, 10465, 10472, 10473Brooklyn Central Brooklyn 11212, 11213, 11216, 11233, 11238 Southwest Brooklyn 11209, 11214, 11228 Borough Park 11204, 11218, 11219, 11230 Canarsie and Flatlands 11234, 11236, 11239 Southern Brooklyn 11223, 11224, 11229, 11235 Northwest Brooklyn 11201, 11205, 11215, 11217, 11231 Flatbush 11203, 11210, 11225, 11226 East New York and New Lots 11207, 11208 Greenpoint 11211, 11222 Sunset Park 11220, 11232 Bushwick and Williamsburg 11206, 11221, 11237Manhattan Central Harlem 10026, 10027, 10030, 10037, 10039 Chelsea and Clinton 10001, 10011, 10018, 10019, 10020, 10036 East Harlem 10029, 10035 Gramercy Park and Murray Hill 10010, 10016, 10017, 10022 Greenwich Village and Soho

In [12]:
import re

In [13]:
zip_string = re.sub("\D", "", zip_string)

In [14]:
zip_string

'10453104571046010458104671046810451104521045610454104551045910474104631047110466104691047010475104611046210464104651047210473112121121311216112331123811209112141122811204112181121911230112341123611239112231122411229112351120111205112151121711231112031121011225112261120711208112111122211220112321120611221112371002610027100301003710039100011001110018100191002010036100291003510010100161001710022100121001310014100041000510006100071003810280100021000310009100211002810044100651007510128100231002410025100311003210033100341004011361113621136311364113541135511356113571135811359113601136511366113671141211423114321143311434114351143611101111021110311104111051110611374113751137911385116911169211693116941169511697110041100511411114131142211426114271142811429114141141511416114171141811419114201142111368113691137011372113731137711378103021030310310103061030710308103091031210301103041030510314'

In [15]:
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))
zip_list = list(chunkstring(zip_string, 5))

In [16]:
zip_list

['10453',
 '10457',
 '10460',
 '10458',
 '10467',
 '10468',
 '10451',
 '10452',
 '10456',
 '10454',
 '10455',
 '10459',
 '10474',
 '10463',
 '10471',
 '10466',
 '10469',
 '10470',
 '10475',
 '10461',
 '10462',
 '10464',
 '10465',
 '10472',
 '10473',
 '11212',
 '11213',
 '11216',
 '11233',
 '11238',
 '11209',
 '11214',
 '11228',
 '11204',
 '11218',
 '11219',
 '11230',
 '11234',
 '11236',
 '11239',
 '11223',
 '11224',
 '11229',
 '11235',
 '11201',
 '11205',
 '11215',
 '11217',
 '11231',
 '11203',
 '11210',
 '11225',
 '11226',
 '11207',
 '11208',
 '11211',
 '11222',
 '11220',
 '11232',
 '11206',
 '11221',
 '11237',
 '10026',
 '10027',
 '10030',
 '10037',
 '10039',
 '10001',
 '10011',
 '10018',
 '10019',
 '10020',
 '10036',
 '10029',
 '10035',
 '10010',
 '10016',
 '10017',
 '10022',
 '10012',
 '10013',
 '10014',
 '10004',
 '10005',
 '10006',
 '10007',
 '10038',
 '10280',
 '10002',
 '10003',
 '10009',
 '10021',
 '10028',
 '10044',
 '10065',
 '10075',
 '10128',
 '10023',
 '10024',
 '10025',


In [17]:
zip_codes = list(map(int, zip_list))

In [18]:
zip_codes

[10453,
 10457,
 10460,
 10458,
 10467,
 10468,
 10451,
 10452,
 10456,
 10454,
 10455,
 10459,
 10474,
 10463,
 10471,
 10466,
 10469,
 10470,
 10475,
 10461,
 10462,
 10464,
 10465,
 10472,
 10473,
 11212,
 11213,
 11216,
 11233,
 11238,
 11209,
 11214,
 11228,
 11204,
 11218,
 11219,
 11230,
 11234,
 11236,
 11239,
 11223,
 11224,
 11229,
 11235,
 11201,
 11205,
 11215,
 11217,
 11231,
 11203,
 11210,
 11225,
 11226,
 11207,
 11208,
 11211,
 11222,
 11220,
 11232,
 11206,
 11221,
 11237,
 10026,
 10027,
 10030,
 10037,
 10039,
 10001,
 10011,
 10018,
 10019,
 10020,
 10036,
 10029,
 10035,
 10010,
 10016,
 10017,
 10022,
 10012,
 10013,
 10014,
 10004,
 10005,
 10006,
 10007,
 10038,
 10280,
 10002,
 10003,
 10009,
 10021,
 10028,
 10044,
 10065,
 10075,
 10128,
 10023,
 10024,
 10025,
 10031,
 10032,
 10033,
 10034,
 10040,
 11361,
 11362,
 11363,
 11364,
 11354,
 11355,
 11356,
 11357,
 11358,
 11359,
 11360,
 11365,
 11366,
 11367,
 11412,
 11423,
 11432,
 11433,
 11434,
 11435,


In [19]:
len(zip_codes)

178

In [20]:
df = df[df.zipcode.isin(zip_codes)]
df

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
95820,36,NY,10001,1,3760.0,2910.0,360.0,400.0,2250.0,4690.0,...,2040.0,2158.0,0.0,0.0,0.0,0.0,750.0,766.0,2510.0,4418.0
95821,36,NY,10001,2,2430.0,1790.0,240.0,330.0,1370.0,3270.0,...,2240.0,7569.0,0.0,0.0,0.0,0.0,510.0,1187.0,1810.0,4285.0
95822,36,NY,10001,3,1930.0,1520.0,200.0,150.0,1110.0,2450.0,...,1910.0,14770.0,0.0,0.0,0.0,0.0,450.0,1397.0,1420.0,3574.0
95823,36,NY,10001,4,1340.0,1070.0,160.0,70.0,770.0,1690.0,...,1330.0,16902.0,0.0,0.0,0.0,0.0,310.0,1209.0,980.0,3268.0
95824,36,NY,10001,5,2480.0,1790.0,540.0,70.0,1520.0,3390.0,...,2470.0,60809.0,110.0,28.0,40.0,42.0,560.0,3816.0,1770.0,7787.0
95825,36,NY,10001,6,2370.0,1230.0,1040.0,50.0,1770.0,4220.0,...,2370.0,438199.0,1880.0,6183.0,1680.0,9464.0,1190.0,38100.0,830.0,21520.0
95826,36,NY,10002,1,22380.0,13110.0,5790.0,3170.0,16530.0,36580.0,...,9890.0,9437.0,0.0,0.0,0.0,0.0,4240.0,3503.0,15660.0,34147.0
95827,36,NY,10002,2,8260.0,4700.0,1890.0,1510.0,5260.0,14590.0,...,7120.0,21290.0,0.0,0.0,0.0,0.0,1640.0,3358.0,6480.0,16415.0
95828,36,NY,10002,3,4690.0,3170.0,710.0,680.0,2750.0,7060.0,...,4630.0,33461.0,0.0,0.0,0.0,0.0,1120.0,3293.0,3470.0,8709.0
95829,36,NY,10002,4,2690.0,1820.0,480.0,300.0,1570.0,3990.0,...,2680.0,33122.0,0.0,0.0,0.0,0.0,590.0,2489.0,2040.0,5930.0


We now have the zip codes we're looking for, and have limited our dataframe to these zip codes.

In [21]:
df.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
95820,36,NY,10001,1,3760.0,2910.0,360.0,400.0,2250.0,4690.0,...,2040.0,2158.0,0.0,0.0,0.0,0.0,750.0,766.0,2510.0,4418.0
95821,36,NY,10001,2,2430.0,1790.0,240.0,330.0,1370.0,3270.0,...,2240.0,7569.0,0.0,0.0,0.0,0.0,510.0,1187.0,1810.0,4285.0
95822,36,NY,10001,3,1930.0,1520.0,200.0,150.0,1110.0,2450.0,...,1910.0,14770.0,0.0,0.0,0.0,0.0,450.0,1397.0,1420.0,3574.0
95823,36,NY,10001,4,1340.0,1070.0,160.0,70.0,770.0,1690.0,...,1330.0,16902.0,0.0,0.0,0.0,0.0,310.0,1209.0,980.0,3268.0
95824,36,NY,10001,5,2480.0,1790.0,540.0,70.0,1520.0,3390.0,...,2470.0,60809.0,110.0,28.0,40.0,42.0,560.0,3816.0,1770.0,7787.0


In [22]:
df = df.iloc[:, [2,3,4]]
df

,zipcode,agi_stub,N1
95820,10001,1,3760.0
95821,10001,2,2430.0
95822,10001,3,1930.0
95823,10001,4,1340.0
95824,10001,5,2480.0
95825,10001,6,2370.0
95826,10002,1,22380.0
95827,10002,2,8260.0
95828,10002,3,4690.0
95829,10002,4,2690.0


In [23]:
df = df.reset_index(drop=True)

In [24]:
df = df.pivot(index='zipcode', columns='agi_stub', values='N1')
df

agi_stub,1,2,3,4,5,6
zipcode,,,,,,
10001,3760.0,2430.0,1930.0,1340.0,2480.0,2370.0
10002,22380.0,8260.0,4690.0,2690.0,3530.0,1640.0
10003,5320.0,4020.0,4340.0,3340.0,6580.0,6450.0
10004,340.0,260.0,290.0,200.0,520.0,880.0
10005,750.0,750.0,910.0,660.0,1360.0,1730.0
10006,300.0,320.0,360.0,260.0,650.0,540.0
10007,440.0,280.0,230.0,220.0,570.0,1590.0
10009,10130.0,6860.0,5800.0,3710.0,5100.0,2110.0
10010,2650.0,2430.0,2570.0,1950.0,3950.0,3930.0


We're interested in the zip code, size range of adjusted gross income ("agi_stub" column), and the number of household filings per range of adjusted gross income. Our dataframe now shows just this information for the zip codes in NYC.

In [25]:
df['Total_Returns'] = df.sum(axis=1)
df

agi_stub,1,2,3,4,5,6,Total_Returns
zipcode,,,,,,,
10001,3760.0,2430.0,1930.0,1340.0,2480.0,2370.0,14310.0
10002,22380.0,8260.0,4690.0,2690.0,3530.0,1640.0,43190.0
10003,5320.0,4020.0,4340.0,3340.0,6580.0,6450.0,30050.0
10004,340.0,260.0,290.0,200.0,520.0,880.0,2490.0
10005,750.0,750.0,910.0,660.0,1360.0,1730.0,6160.0
10006,300.0,320.0,360.0,260.0,650.0,540.0,2430.0
10007,440.0,280.0,230.0,220.0,570.0,1590.0,3330.0
10009,10130.0,6860.0,5800.0,3710.0,5100.0,2110.0,33710.0
10010,2650.0,2430.0,2570.0,1950.0,3950.0,3930.0,17480.0


We now show the total number of filings for each zip code. We'll use this to determine the makeup of each household income range for all zip codes, and then score each zip code's relative household wealth on a scale of 1-6.

In [26]:
for i in range(1,7):
    col_name = "Bracket_"+str(i)+"_Percentage"
    df[col_name] = df[i]/df['Total_Returns']
df

agi_stub,1,2,3,4,5,6,Total_Returns,Bracket_1_Percentage,Bracket_2_Percentage,Bracket_3_Percentage,Bracket_4_Percentage,Bracket_5_Percentage,Bracket_6_Percentage
zipcode,,,,,,,,,,,,,
10001,3760.0,2430.0,1930.0,1340.0,2480.0,2370.0,14310.0,0.262753,0.169811,0.134871,0.093641,0.173305,0.165618
10002,22380.0,8260.0,4690.0,2690.0,3530.0,1640.0,43190.0,0.518176,0.191248,0.108590,0.062283,0.081732,0.037972
10003,5320.0,4020.0,4340.0,3340.0,6580.0,6450.0,30050.0,0.177038,0.133777,0.144426,0.111148,0.218968,0.214642
10004,340.0,260.0,290.0,200.0,520.0,880.0,2490.0,0.136546,0.104418,0.116466,0.080321,0.208835,0.353414
10005,750.0,750.0,910.0,660.0,1360.0,1730.0,6160.0,0.121753,0.121753,0.147727,0.107143,0.220779,0.280844
10006,300.0,320.0,360.0,260.0,650.0,540.0,2430.0,0.123457,0.131687,0.148148,0.106996,0.267490,0.222222
10007,440.0,280.0,230.0,220.0,570.0,1590.0,3330.0,0.132132,0.084084,0.069069,0.066066,0.171171,0.477477
10009,10130.0,6860.0,5800.0,3710.0,5100.0,2110.0,33710.0,0.300504,0.203500,0.172056,0.110056,0.151290,0.062593
10010,2650.0,2430.0,2570.0,1950.0,3950.0,3930.0,17480.0,0.151602,0.139016,0.147025,0.111556,0.225973,0.224828


In [27]:
df['average'] = [0]*len(df)
for x in range(7,13):
    df['average'] += df.iloc[:,x]*(x-6)
df.sort_values(by='average')

agi_stub,1,2,3,4,5,6,Total_Returns,Bracket_1_Percentage,Bracket_2_Percentage,Bracket_3_Percentage,Bracket_4_Percentage,Bracket_5_Percentage,Bracket_6_Percentage,average
zipcode,,,,,,,,,,,,,,
10453,23970.0,9430.0,3030.0,880.0,560.0,0.0,37870.0,0.632955,0.249010,0.080011,0.023237,0.014787,0.000000,1.537893
10454,10000.0,4070.0,1260.0,330.0,230.0,20.0,15910.0,0.628536,0.255814,0.079195,0.020742,0.014456,0.001257,1.540541
10456,25320.0,10540.0,3190.0,950.0,520.0,20.0,40540.0,0.624568,0.259990,0.078688,0.023434,0.012827,0.000493,1.541441
10457,20850.0,8230.0,2670.0,800.0,460.0,30.0,33040.0,0.631053,0.249092,0.080811,0.024213,0.013923,0.000908,1.543584
11220,37090.0,9750.0,3620.0,1680.0,1740.0,390.0,54270.0,0.683435,0.179657,0.066704,0.030956,0.032062,0.007186,1.570112
10452,22230.0,9340.0,3330.0,960.0,500.0,30.0,36390.0,0.610882,0.256664,0.091509,0.026381,0.013740,0.000824,1.577906
11355,34930.0,8220.0,3480.0,1820.0,1880.0,350.0,50680.0,0.689227,0.162194,0.068666,0.035912,0.037096,0.006906,1.590174
10474,3010.0,1380.0,470.0,130.0,70.0,0.0,5060.0,0.594862,0.272727,0.092885,0.025692,0.013834,0.000000,1.590909
10455,10630.0,4830.0,1660.0,510.0,270.0,0.0,17900.0,0.593855,0.269832,0.092737,0.028492,0.015084,0.000000,1.601117


In [28]:
df.to_csv('zips_and_scores.csv', sep=',')

We'll import the .csv we created above to our map plot in order to represent this visually.

That's it!